In [2]:
import re
import numpy as np
import pandas as pd
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from pandas.api.types import CategoricalDtype
from scipy.stats import chi2, norm

**Calculate Mean and StD**

In [12]:
res = [0.877,
       0.9067,
       0.9406]

mean_res = np.mean(res)
std_res = np.std(res)

print(mean_res)
print(std_res)

0.9081
0.025983456275099348


**Get Mean Accuracies for different setups**

In [16]:
path = "vlcs_tta_results_all_domains_20250820_115337.txt"

with open(path, encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

records = []
current_domain, current_mode, current_seed, current_target = None, None, None, None
in_seed_block = False

re_domain   = re.compile(r"^=== TEST DOMAIN:\s+([A-Za-z0-9_]+)\s+===")
re_mode     = re.compile(r"^Mode:\s*([A-Z_0-9]+)\s*$")
re_seed     = re.compile(r"^Seed:\s*(\d+)\s*$")
re_target   = re.compile(r"^\s*Target\s+([A-Za-z0-9_]+):\s*$")
re_accuracy = re.compile(r"^\s*Accuracy:\s*([0-9]*\.[0-9]+)\s*$")

def norm_domain(d): return d.lower().replace("_", " ")
def norm_mode(m): return m.lower()

for line in lines:
    m = re_domain.match(line)
    if m:
        current_domain = norm_domain(m.group(1))
        continue
    m = re_mode.match(line)
    if m:
        current_mode = norm_mode(m.group(1))
        continue
    m = re_seed.match(line)
    if m:
        current_seed = int(m.group(1))
        in_seed_block = True
        continue
    if in_seed_block:
        mt = re_target.match(line)
        if mt:
            current_target = mt.group(1).lower()
            continue
        ma = re_accuracy.match(line)
        if ma and current_target is not None:
            acc = float(ma.group(1))
            records.append({
                "domain": current_domain,
                "mode": current_mode,
                "seed": current_seed,
                "target": current_target,
                "accuracy": acc,
            })

df = pd.DataFrame(records)

def mean_excl_original(group):
    vals = [v for t, v in zip(group["target"], group["accuracy"]) if t != "original"]
    return sum(vals)/len(vals) if vals else float("nan")

agg = (
    df.groupby(["domain","mode","seed"])
      .apply(lambda g: mean_excl_original(g.reset_index(drop=True)))
      .reset_index(name="mean_accuracy")
)

for _, row in agg.iterrows():
    print(f"{row['domain']}, {row['mode']}, seed {row['seed']}: {row['mean_accuracy']:}")


caltech101, average, seed 0: 0.6148
caltech101, average, seed 7: 0.6148
caltech101, average, seed 42: 0.6148
caltech101, selective_0_1, seed 0: 0.9644333333333334
caltech101, selective_0_1, seed 7: 0.9404
caltech101, selective_0_1, seed 42: 0.9686666666666667
caltech101, selective_0_2, seed 0: 0.8859666666666666
caltech101, selective_0_2, seed 7: 0.8379
caltech101, selective_0_2, seed 42: 0.9225
caltech101, selective_0_3, seed 0: 0.6148
caltech101, selective_0_3, seed 7: 0.43883333333333335
caltech101, selective_0_3, seed 42: 0.6148
caltech101, selective_1_2, seed 0: 0.8622
caltech101, selective_1_2, seed 7: 0.8158
caltech101, selective_1_2, seed 42: 0.9088333333333334
caltech101, selective_1_3, seed 0: 0.6148
caltech101, selective_1_3, seed 7: 0.43883333333333335
caltech101, selective_1_3, seed 42: 0.6148
caltech101, selective_2_3, seed 0: 0.6148
caltech101, selective_2_3, seed 7: 0.43883333333333335
caltech101, selective_2_3, seed 42: 0.6148
caltech101, single_0, seed 0: 0.9767
calte

/tmp/ipykernel_3388907/272447444.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: mean_excl_original(g.reset_index(drop=True)))


In [ ]:
FILE = Path("vlcs_tta_results_all_domains_20250820_115337.txt")

# %% Parser
test_pat = re.compile(r"^=== TEST DOMAIN:\s+([A-Za-z0-9_]+)\s+===")
mode_pat  = re.compile(r"^Mode:\s+([A-Z0-9_]+)")
seed_pat  = re.compile(r"^Seed:\s+(\d+)")
tgt_pat = re.compile(r"^\s*Target\s+([A-Za-z0-9_]+):\s*$")
acc_pat   = re.compile(r"^\s*Accuracy:\s*([0-9]*\.?[0-9]+)\s*$")

def normalize_domain(name):
    name = name.strip()
    name = name.lower()
    name = name.replace(" ", "_")
    return name

rows = []  # dict(test_domain, mode, seed, target_domain, acc)

cur_test = None
cur_mode = None
cur_seed = None
cur_target = None

with FILE.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip("\n")

        m = test_pat.match(line)
        if m:
            cur_test = normalize_domain(m.group(1))
            cur_mode = None
            cur_seed = None
            cur_target = None
            continue

        m = mode_pat.match(line)
        if m:
            cur_mode = m.group(1).upper()  # f.e. SINGLE_0, SELECTIVE_0_1, AVERAGE
            cur_seed = None
            cur_target = None
            continue

        m = seed_pat.match(line)
        if m:
            cur_seed = int(m.group(1))
            cur_target = None
            continue

        m = tgt_pat.match(line)
        if m:
            cur_target = normalize_domain(m.group(1))
            continue

        m = acc_pat.match(line)
        if m and cur_test and cur_mode and cur_seed is not None and cur_target:
            acc = float(m.group(1))
            rows.append({
                "test_domain": cur_test,
                "mode": cur_mode,
                "seed": cur_seed,
                "target_domain": cur_target,
                "accuracy": acc
            })
            continue

df = pd.DataFrame(rows)

df_no_orig = df[df["target_domain"] != "original"].copy()

agg = (
    df_no_orig
    .groupby(["test_domain", "mode", "target_domain"], as_index=False)
    .agg(mean_acc=("accuracy", "mean"),
         std_acc =("accuracy", "std"),
         n_seeds=("accuracy", "size"))
)

#domain_order = ["art_painting", "cartoon", "photo", "sketch"]
domain_order = ["caltech101", "labelme", "sun09", "voc2007"]

agg["test_domain"] = pd.Categorical(agg["test_domain"], domain_order, ordered=True)
agg["target_domain"] = pd.Categorical(agg["target_domain"], domain_order, ordered=True)
agg = agg.sort_values(["test_domain", "mode", "target_domain"]).reset_index(drop=True)

print("tidy table (one row per test_domain / mode / target_domain):")
display(agg)

def fmt(mean, std):
    if pd.isna(std):
        return f"{mean:.4f} ± nan"
    return f"{mean:.4f} ± {std:.4f}"

pivot = (
    agg
    .assign(mean_std=agg.apply(lambda r: fmt(r["mean_acc"], r["std_acc"]), axis=1))
    .pivot_table(index=["test_domain", "mode"], columns="target_domain", values="mean_std", aggfunc="first")
    .reindex(columns=[d for d in domain_order if d in agg["target_domain"].unique()])
    .sort_index()
)

print("\nPivot (mean ± std):")
display(pivot)

tidy table (one row per test_domain / mode / target_domain):


,test_domain,mode,target_domain,mean_acc,std_acc,n_seeds
0,caltech101,AVERAGE,labelme,0.614800,0.000000,3
1,caltech101,AVERAGE,sun09,0.614800,0.000000,3
2,caltech101,AVERAGE,voc2007,0.614800,0.000000,3
3,caltech101,SELECTIVE_0_1,labelme,0.957100,0.016074,3
4,caltech101,SELECTIVE_0_1,sun09,0.956433,0.017381,3
...,...,...,...,...,...,...
127,voc2007,SINGLE_2,labelme,0.663500,0.063876,3
128,voc2007,SINGLE_2,sun09,0.709033,0.016929,3
129,voc2007,SINGLE_3,caltech101,0.444000,0.000000,3
130,voc2007,SINGLE_3,labelme,0.286000,0.136832,3



Pivot (mean ± std):


/tmp/ipykernel_3338927/892181203.py:92: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  agg


target_domain                   caltech101          labelme            sun09  \
test_domain mode                                                               
caltech101  AVERAGE                    NaN  0.6148 ± 0.0000  0.6148 ± 0.0000   
            SELECTIVE_0_1              NaN  0.9571 ± 0.0161  0.9564 ± 0.0174   
            SELECTIVE_0_2              NaN  0.8582 ± 0.0481  0.8850 ± 0.0460   
            SELECTIVE_0_3              NaN  0.4388 ± 0.3048  0.6148 ± 0.0000   
            SELECTIVE_1_2              NaN  0.8349 ± 0.0582  0.8565 ± 0.0509   
            SELECTIVE_1_3              NaN  0.4388 ± 0.3048  0.6148 ± 0.0000   
            SELECTIVE_2_3              NaN  0.4388 ± 0.3048  0.6148 ± 0.0000   
            SINGLE_0                   NaN  0.9774 ± 0.0056  0.9750 ± 0.0057   
            SINGLE_1                   NaN  0.9628 ± 0.0138  0.9656 ± 0.0134   
            SINGLE_2                   NaN  0.8758 ± 0.0456  0.9008 ± 0.0402   
            SINGLE_3                   NaN  0.4388 ± 0.3048  0.6148 ± 0.0000   
labelme     AVERAGE        0.4657 ± 0.0000              NaN  0.4657 ± 0.0000   
            SELECTIVE_0_1  0.6324 ± 0.0008              NaN  0.6207 ± 0.0014   
            SELECTIVE_0_2  0.6215 ± 0.0167              NaN  0.5901 ± 0.0272   
            SELECTIVE_0_3  0.4657 ± 0.0000              NaN  0.4657 ± 0.0000   
            SELECTIVE_1_2  0.6277 ± 0.0163              NaN  0.5906 ± 0.0281   
            SELECTIVE_1_3  0.4657 ± 0.0000              NaN  0.4657 ± 0.0000   
            SELECTIVE_2_3  0.4657 ± 0.0000              NaN  0.4657 ± 0.0000   
            SINGLE_0       0.6276 ± 0.0025              NaN  0.6287 ± 0.0025   
            SINGLE_1       0.6393 ± 0.0047              NaN  0.6272 ± 0.0025   
            SINGLE_2       0.6375 ± 0.0166              NaN  0.6101 ± 0.0241   
            SINGLE_3       0.4657 ± 0.0000              NaN  0.4657 ± 0.0000   
sun09       AVERAGE        0.3851 ± 0.0000  0.3851 ± 0.0000              NaN   
            SELECTIVE_0_1  0.6922 ± 0.0041  0.6969 ± 0.0093              NaN   
            SELECTIVE_0_2  0.5586 ± 0.0488  0.5750 ± 0.0034              NaN   
            SELECTIVE_0_3  0.3851 ± 0.0000  0.2840 ± 0.0000              NaN   
            SELECTIVE_1_2  0.5433 ± 0.0463  0.5693 ± 0.0058              NaN   
            SELECTIVE_1_3  0.3851 ± 0.0000  0.2840 ± 0.0000              NaN   
            SELECTIVE_2_3  0.3851 ± 0.0000  0.2840 ± 0.0000              NaN   
            SINGLE_0       0.6979 ± 0.0138  0.7035 ± 0.0153              NaN   
            SINGLE_1       0.6984 ± 0.0052  0.7048 ± 0.0120              NaN   
            SINGLE_2       0.5951 ± 0.0489  0.5798 ± 0.0013              NaN   
            SINGLE_3       0.3851 ± 0.0000  0.2840 ± 0.0000              NaN   
voc2007     AVERAGE        0.4440 ± 0.0000  0.4440 ± 0.0000  0.4440 ± 0.0000   
            SELECTIVE_0_1  0.7695 ± 0.0082  0.7669 ± 0.0079  0.7571 ± 0.0093   
            SELECTIVE_0_2  0.7035 ± 0.0369  0.6384 ± 0.0588  0.6910 ± 0.0097   
            SELECTIVE_0_3  0.4440 ± 0.0000  0.2860 ± 0.1368  0.4440 ± 0.0000   
            SELECTIVE_1_2  0.6932 ± 0.0386  0.6286 ± 0.0572  0.6787 ± 0.0096   
            SELECTIVE_1_3  0.4440 ± 0.0000  0.2860 ± 0.1368  0.4440 ± 0.0000   
            SELECTIVE_2_3  0.4440 ± 0.0000  0.2860 ± 0.1368  0.4440 ± 0.0000   
            SINGLE_0       0.7730 ± 0.0106  0.7690 ± 0.0056  0.7687 ± 0.0071   
            SINGLE_1       0.7770 ± 0.0085  0.7703 ± 0.0086  0.7650 ± 0.0088   
            SINGLE_2       0.7292 ± 0.0352  0.6635 ± 0.0639  0.7090 ± 0.0169   
            SINGLE_3       0.4440 ± 0.0000  0.2860 ± 0.1368  0.4440 ± 0.0000   

target_domain                      voc2007  
test_domain mode                            
caltech101  AVERAGE        0.6148 ± 0.0000  
            SELECTIVE_0_1  0.9600 ± 0.0125  
            SELECTIVE_0_2  0.9032 ± 0.0336  
            SELECTIVE_0_3  0.6148 ± 0.0000  
            SELECTIVE_1_2  0.8954 ± 0.0315  
    

**EXPERIMENT: Linear Mixed Model for Statistical Evaluation**

In [3]:
df = pd.read_csv("vlcs_results.csv")

df = df.rename(columns={
    "domain": "domain",
    "seed": "seed",
    "approach": "approach",
    "mode": "mode",
    "mean accuracy": "accuracy"
})
df["accuracy"] = pd.to_numeric(df["accuracy"], errors="coerce")
df = df.dropna(subset=["accuracy", "domain", "seed", "approach", "mode"]).copy()

# categories
df["approach"] = pd.Categorical(df["approach"], categories=["MixStyle", "TTA"])
df.loc[df["approach"] == "MixStyle", "mode"] = "base"
df["mode"] = pd.Categorical(df["mode"])

# domain × seed id for random effects
df["domain_seed"] = df["domain"].astype(str) + "_" + df["seed"].astype(str)

# ----------------------
# 1) TTA-only
# ----------------------
df_tta = df[df["approach"] == "TTA"].copy()
df_tta["mode"] = df_tta["mode"].cat.remove_unused_categories()

vc_tta = {"seed": "0 + C(domain_seed)"}
model_tta = smf.mixedlm(
    "accuracy ~ C(mode)",
    data=df_tta,
    groups=df_tta["domain"],
    vc_formula=vc_tta
)
res_tta = model_tta.fit(method="lbfgs", reml=False, maxiter=200)
print("\n=== TTA-only: Mode-Effekte ===")
print(res_tta.summary())

# ----------------------
# 2) MixStyle (base) vs TTA modes
# ----------------------
best_modes = ["single_0", "single_1", "single_2", "selective_0_1", "selective_0_2", "selective_1_2"]

df_compare = df[
    ((df["approach"] == "MixStyle") & (df["mode"] == "base"))
    | ((df["approach"] == "TTA") & (df["mode"].isin(best_modes)))
].copy()

df_compare["mode"] = pd.Categorical(
    df_compare["mode"],
    categories=["base"] + [m for m in best_modes if m in df_compare["mode"].unique()]
)

vc_compare = {"seed": "0 + C(domain_seed)"}

model_compare = smf.mixedlm(
    "accuracy ~ C(mode, Treatment(reference='base'))",
    data=df_compare,
    groups=df_compare["domain"],
    vc_formula=vc_compare
)

try:
    res_compare = model_compare.fit(method="lbfgs", reml=False, maxiter=200)
except Exception:
    model_compare_simple = smf.mixedlm(
        "accuracy ~ C(mode, Treatment(reference='base'))",
        data=df_compare,
        groups=df_compare["domain"]
    )
    res_compare = model_compare_simple.fit(method="lbfgs", reml=False, maxiter=200)

print("\n=== MixStyle (base) vs. einzelne TTA-Modi ===")
print(res_compare.summary())

# ----------------------
# 3) Effekt-Tabelle extrahieren
# ----------------------
def fixed_effects_table(res):
    ci = res.conf_int()
    return pd.DataFrame({
        "coef": res.params,
        "ci_low": ci[0],
        "ci_high": ci[1]
    })

print("\nFixed Effects (TTA-only):\n", fixed_effects_table(res_tta))
print("\nFixed Effects (MixStyle vs. beste TTA):\n", fixed_effects_table(res_compare))

# ----------------------
# 4) MixStyle (base) vs domain x mode
# ----------------------

modes_available = [m for m in df_compare["mode"].cat.categories if m != "base"]

# Wide-Table: Zeilen = (domain, seed), Spalten = mode, Werte = accuracy
wide = (
    df_compare.pivot_table(index=["domain", "seed"], columns="mode", values="accuracy", aggfunc="mean")
    .reindex(columns=["base"] + modes_available)
)

results = []
for dom in wide.index.get_level_values("domain").unique():
    sub = wide.loc[dom]  # DataFrame: index=seed, cols=base + modes
    for mode in modes_available:
        diffs = (sub[mode] - sub["base"]).dropna()  # nur Seeds mit beiden Werten
        n = diffs.shape[0]
        if n == 0:
            mean_diff = np.nan
            sd = se = ci_low = ci_high = np.nan
        else:
            mean_diff = diffs.mean()
            sd = diffs.std(ddof=1) if n > 1 else 0.0
            se = sd / np.sqrt(n) if n > 1 else 0.0
            # 95%-CI mit t-Verteilung (bei n=1 kein CI sinnvoll)
            if n > 1:
                t_crit = stats.t.ppf(0.975, df=n-1)
                ci_low = mean_diff - t_crit * se
                ci_high = mean_diff + t_crit * se
            else:
                ci_low = ci_high = np.nan

        results.append({
            "domain": dom,
            "mode": mode,
            "n_seed_pairs": n,
            "mean_improvement_over_base": mean_diff,
            "sd_diff": sd,
            "se_diff": se,
            "ci95_low": ci_low,
            "ci95_high": ci_high,
        })

improvement_table = pd.DataFrame(results).sort_values(["domain", "mode"]).reset_index(drop=True)
print("\n=== Domänen × Modus: Verbesserung ggü. base (gepaart pro Seed) ===")
print(improvement_table)



=== TTA-only: Mode-Effekte ===
               Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     accuracy
No. Observations:      131         Method:                 ML      
No. Groups:            4           Scale:                  0.0022  
Min. group size:       32          Log-Likelihood:         192.3458
Max. group size:       33          Converged:              Yes     
Mean group size:       32.8                                        
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 0.477    0.031 15.456 0.000  0.417  0.538
C(mode)[T.selective_0_1]  0.284    0.020 14.520 0.000  0.245  0.322
C(mode)[T.selective_0_2]  0.213    0.019 11.167 0.000  0.176  0.250
C(mode)[T.selective_0_3] -0.036    0.019 -1.902 0.057 -0.074  0.001
C(mode)[T.selective_1_2]  0.204

/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/tmp/ipykernel_3396556/1212942863.py:97: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_compare.pivot_table(index=["domain", "seed"], columns="mode", values="accuracy", aggfunc="mean")


**paired t-test**

In [9]:
# 1) Daten für MixStyle und single_1
df_pair = df[df["mode"].isin(["base", "single_1"])].copy()

# 2) Mittelwert pro Domäne & Modus (Seeds aggregieren)
df_mean = df_pair.groupby(["domain", "mode"], observed=True)["accuracy"].mean().unstack("mode")

# 3) Paired t-Test (Differenz pro Domäne: single_1 - base)
t_stat, p_val = stats.ttest_rel(df_mean["single_1"], df_mean["base"])

print("=== Paired t-Test: MixStyle (base) vs. TTA (single_1) ===")
print("t-Statistic:", t_stat)
print("p-Value:", p_val)
print("\nMean Accuracy (over seeds):")
print(df_mean)
print("\nMean Difference (single_1 - base):", (df_mean["single_1"] - df_mean["base"]).mean())


=== Paired t-Test: MixStyle (base) vs. TTA (single_1) ===
t-Statistic: 2.100453566566424
p-Value: 0.1265105657848677

Mean Accuracy (over seeds):
mode              base  single_1
domain                          
art painting  0.853779  0.874667
cartoon       0.777022  0.795733
photo         0.969928  0.968633
sketch        0.738481  0.787967

Mean Difference (single_1 - base): 0.021947249330748597
